In [1]:
# import libraries
import pandas as pd
import pyarrow.parquet as pq
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
#import geopandas as gpd
# import osmnx as ox
import json
from datetime import datetime, timedelta, date
import os
import time
import sys
import requests
# import sidetable
import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.tracebacklimit = 0 # turn off long tracebacks on error messages
#sys.tracebacklimit = None # use to turn tracebacks back on if needed

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [4]:
#extra columns

#                 'therapeutic_bamlanivimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_bamlanivimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_casirivimab_plus_imdevimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_casirivimab_plus_imdevimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_bamlanivimab_plus_etesevimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_bamlanivimab_plus_etesevimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_sotrovimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_sotrovimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_evusheld_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_evusheld_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_molnupiravir_number_ of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_molnupiravir_number_ of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_paxlovid_number_ of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_paxlovid_number_ of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_bebtelovimab_number_of_residents_treated from_stock_stored_at_this_facility',
#                'therapeutic_bebtelovimab_number_of_residents_treated from_stock_stored_at_another_facility',

In [5]:
pfile = pq.read_table("data/nh_all_Jun27_21_Dec18_22_concat.parquet.gzip")
print("Column names: {}".format(pfile.column_names))
print("Schema: {}".format(pfile.schema))

Column names: ['week_ending', 'federal_provider_number', 'provider_name', 'provider_address', 'provider_city', 'provider_state', 'provider_zip_code', 'provider_phone_number', 'county', 'submitted_data', 'passed_quality_assurance_check', 'residents_weekly_admissions_covid_19', 'residents_total_admissions_covid_19', 'residents_weekly_confirmed_covid_19', 'residents_total_confirmed_covid_19', 'residents_weekly_all_deaths', 'residents_total_all_deaths', 'residents_weekly_covid_19_deaths', 'residents_total_covid_19_deaths', 'number_of_all_beds', 'total_number_of_occupied_beds', 'staff_weekly_confirmed_covid_19', 'staff_total_confirmed_covid_19', 'staff_weekly_covid_19_deaths', 'staff_total_covid_19_deaths', 'Staff_Shortage', 'shortage_of_nursing_staff', 'shortage_of_clinical_staff', 'shortage_of_aides', 'shortage_of_other_staff', 'weekly_resident_confirmed_covid_19_cases_per_1_000_residents', 'weekly_resident_covid_19_deaths_per_1_000_residents', 'total_resident_confirmed_covid_19_cases_per

In [6]:
# set keep only needed cols
needed_cols = ['week_ending',
                'federal_provider_number',
                'provider_name',
                'provider_address',
                'provider_city',
                'provider_state',
                'provider_zip_code',
                'provider_phone_number',
                'county',
                'submitted_data',
                'passed_quality_assurance_check',
                'residents_weekly_confirmed_covid_19',
                'residents_weekly_all_deaths',
                'residents_weekly_covid_19_deaths',
                'number_of_all_beds',
                'total_number_of_occupied_beds',
                'staff_weekly_confirmed_covid_19',
                'staff_weekly_covid_19_deaths',
                'shortage_of_nursing_staff',
                'shortage_of_clinical_staff',
                'shortage_of_aides',
                'weekly_resident_confirmed_covid_19_cases_per_1_000_residents',
                'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result',
                'number_of_residents_with_new_influenza',
                'number_of_staff_and_or_personnel_with_new_influenza',
                'Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time',
                'recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time',
                'percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time',
                'recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time',
                'recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
                'percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
                'Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines',
                'Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines',
                'Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines',
                'number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time',
                'recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
                'percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
                'Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines',
                'Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines',
                'Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines',
                'Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines']



In [ ]:
nh_latest.info()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



NameError: name 'nh_latest' is not defined

During handling of the above exception, another exception occurred:

AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
NameError: name 'nh_latest' is not defined

During handling of the above exception, another exception occurred:

AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

TypeError: object of type 'NoneType' has no len()

During handling of the above exception, another exception occurred:

AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
NameError: name 'nh_latest' is not defined

During handling of the above exception, another exception occurred:

AttributeError: 'NameError' object has no attribute '_render_traceback_'

During h

In [7]:
nh_latest = pfile.to_pandas()

ERROR! Session/line number was not unique in database. History logging moved to new session 462


In [8]:
nh_latest_sub = nh_latest[needed_cols]

In [9]:
nh_latest_sub.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2022-12-04,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,1,0,57,52,1,0,,,,0,1,0,0,Y,53,49,4,,92.453,92.453,81,70,10,86.42,87.5,,48,,97.959,47,,88.679,24,,34.286,24,,29.63,34.286
1,2022-12-04,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,

In [10]:
nh_latest_sub.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
1184371,2022-11-27,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,N,0,0,0,12,2,0,0,,,,0,0,0,0,Y,2,0,2,,0,0,12,10,2,83.333,83.333,,0,,0,0,,0,0,,0,1,,8.333,10
1184372,2022-11-27,745004,THE BRIXTON AT HORSESHOE BAY,15101 WEST FM 2147,HORSESHOE BAY,TX,78657,7135531321,Burnet,Y,Y,0,0,0,120,48,0,0,,,,0,0,0,0,Y,48,45,3,,93.

In [11]:
list(nh_latest_sub.columns)

['week_ending',
 'federal_provider_number',
 'provider_name',
 'provider_address',
 'provider_city',
 'provider_state',
 'provider_zip_code',
 'provider_phone_number',
 'county',
 'submitted_data',
 'passed_quality_assurance_check',
 'residents_weekly_confirmed_covid_19',
 'residents_weekly_all_deaths',
 'residents_weekly_covid_19_deaths',
 'number_of_all_beds',
 'total_number_of_occupied_beds',
 'staff_weekly_confirmed_covid_19',
 'staff_weekly_covid_19_deaths',
 'shortage_of_nursing_staff',
 'shortage_of_clinical_staff',
 'shortage_of_aides',
 'weekly_resident_confirmed_covid_19_cases_per_1_000_residents',
 'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result',
 'number_of_residents_with_new_influenza',
 'number_of_staff_and_or_personnel_with_new_influenza',
 'Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel',
 'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week',
 'number_of_residents_staying_in_this_facility_

In [12]:
nh_latest_sub.sort_values(by=['week_ending']).head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
45748,2021-06-27,045189,SOMERSET SENIOR LIVING AT PINE HILLS,900 MAGNOLIA RD,CAMDEN,AR,71701,8708366833,Ouachita,Y,Y,0,0,0,106,57,0,0,N,N,N,0,0,0,0,,59,53,5,,89.831,89.831,91,32,52,35.165,35.165,,,,0,,,0,,,0,,,0,0
55489,2021-06-27,365939,EDGEWOOD MANOR OF WELLSTON,405 NORTH PARK AVENUE,WELLSTON,OH,45692,7403845611,Jackson,Y,Y,0,0,0,50,38,0,0,N,N,N,0,0,0,0,,

In [13]:
nh_latest_sub.sort_values(by=['week_ending']).tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
35256,2022-12-18,185295,"HARBORVIEW DOVER, LLC",112 DOVER DRIVE,GEORGETOWN,KY,40324,5028639529,Scott,Y,Y,3,1,0,85,80,0,0,,,,37.5,0,0,0,Y,80,74,5,92.5,92.5,92.5,113,75,34,66.372,66.372,66.372,69,93.243,93.243,66,82.5,82.5,32,42.667,42.667,32,28.319,28.319,42.667
35257,2022-12-18,185297,SANSBURY CARE CENTER,2625 BARDSTOWN ROAD,SAINT CATHARINE,KY,40061,8593363

In [14]:
#check for dup rows
nh_latest_sub.loc[nh_latest_sub.duplicated(),]

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines


In [15]:
# rename long cols
nh_latest_sub = nh_latest_sub.rename({
    'provider_zip_code':'provider_zip',
    'passed_quality_assurance_check':'data_passed_qa',
    'residents_weekly_confirmed_covid_19':'res_wk_confirmed_c19',
    'residents_weekly_all_deaths': 'res_wk_all_deaths',
    'residents_weekly_covid_19_deaths':'res_wk_c19_deaths',
    'number_of_all_beds':'num_all_beds',
    'total_number_of_occupied_beds':'num_occ_beds',
    'staff_weekly_confirmed_covid_19':'staff_wk_conf_c19',
     'staff_weekly_covid_19_deaths':'staff_wk_c19_deaths',
    'shortage_of_nursing_staff':'short_nrsg_staff',
    'shortage_of_aides':'short_aides',
    'weekly_resident_confirmed_covid_19_cases_per_1_000_residents':'wk_res_confirmed_c19_per_1K',
    'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result':'num_staff_new_pos_c19_test',
    'number_of_residents_with_new_influenza':'num_res_new_flu',
    'number_of_staff_and_or_personnel_with_new_influenza':'num_staff_pers_new_flu',
    'Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel':'can_test_all_res_staff_pers',
    'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week':'num_res_in_facility',
    'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_res_comp_c19_vax_anytime',
    'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_res_decln_c19_vax_anytime',
    'recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time':'rec_pct_res_comp_c19_vax_anytime',
    'percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time':'pct_res_comp_c19_vax_anytime',
    'Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines':'num_res_utd_c19_vax',
    'Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines':'rec_pct_res_utd_c19_vax',
    'Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines':'pct_res_utd_c19_vax',
    'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week':'num_hcp',
    'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_hcp_comp_c19_vax_anytime',
    'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_hcp_decln_c19_vax_anytime',
    'recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time': 'rec_pct_hcp_comp_c19_vax_anytime',
    'percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time': 'pct_hcp_comp_c19_vax_anytime',
    'Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines':'num_hcp_utd_c19_vax',
    'Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines':'rec_pct_hcp_utd_c19_vax',
    'Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines':'pct_hcp_utd_c19_vax',
    'Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines':'pct_hcp_comp_c19_vax_and_utd',
   }, axis=1)

In [16]:
nh_latest_sub.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,shortage_of_clinical_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,rec_pct_hcp_comp_c19_vax_anytime,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,pct_res_utd_c19_vax,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd
0,2022-12-04,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,1,0,57,52,1,0,,,,0,1,0,0,Y,53,49,4,,92.453,92.453,81,70,10,86.42,87.5,,48,,97.959,47,,88.679,24,,34.286,24,,29.63,34.286
1,2022-12-04,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,0,85,79,0,0,,,,0,0,0,0,Y,80,73,5,,91.25,91.25,186,152,30,81.72,82.609,,67,,91.781,59,,73.75,26,,17.105,1,,0.538,0.658
2,2022-12-04,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,0,0,50,37,3,0,,,,0,3,0,0,Y,38,38,0,,100,100,81,77,4,95.062,95.062,,37,,97.368,34,,89.474,56,,72.727,14,,17.284,18.182
3,2022-12-04,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,0,0,90,76,0,0,,,,0,0,0,0,Y,76,74,1,,97.368,97.368,102,99,2,97.059,97.059,,73,,98.649,73,,96.053,80,,80.808,30,,29.412,30.303
4,2022-12-04,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,2,0,103,87,0,0,,,,0,0,0,0,Y,89,82,5,,92.135,92.135,141,117,18,82.979,82.979,,76,,92.683,65,,73.034,75,,64.103,5,,3.546,4.274


### What fields do we actualy care about?  From CMS site: https://data.cms.gov/covid-19/covid-19-nursing-home-data:

* Recent Percentage of Current Residents who Received a Completed COVID-19 Vaccination at Any Time
* Recent Percentage of Current Residents Up to Date with COVID-19 Vaccines
* Recent Percentage of Current Healthcare Personnel who Received a Completed COVID-19 Vaccination at Any Time
* Recent Percentage of Current Healthcare Personnel Up to Date with COVID-19 Vaccines

### Rounding back to the data dictionary for clarity on how these are calculated:
https://data.cms.gov/sites/default/files/2022-11/COVID-19%20Nursing%20Home%20Data%20Dictionary%2011.20.2022.pdf


#### Resident Metrics:
* Recent Percentage of Current Residents who Received a Completed COVID-19 Vaccination at Any Time = The value of "Percentage of Current Residents who Received a Completed COVID-19 Vaccination at Any Time" for the current week, or if blank, for the prior week. Calculated as follows: <b>(Number of Residents Staying in this Facility for At Least 1 Day This Week who Received a Completed COVID-19 Vaccination at Any Time</b> / <b> Number of Residents Staying in this Facility for At Least 1 Day This Week) * 100 </b>
<br>
<br>
* <mark> Recent Percentage of Current Residents Up to Date with COVID-19 Vaccines = The value of "Percentage of Residents who are Up to Date with COVID-19 Vaccines" for the current week, or if blank, for the prior week.
Calculated as follows: <b>(Number of Residents Staying in this Facility
for At Least 1 Day This Week Up to Date with COVID-19 Vaccines</b> /
<b>Number of Residents Staying in this Facility for At Least 1 Day This
Week) * 100</b></mark>
<br>

#### Health Care Personnel Metrics:
* Recent Percentage of Current Healthcare Personnel who Received a Completed COVID-19 Vaccination at Any Time = The value of "Percentage of Current Healthcare Personnel who Received a Completed COVID-19 Vaccination at Any Time" for the current week, or if blank, for the prior week. Calculated as follows: <b> (Number of All Healthcare Personnel Eligible to Work in this Facility for At Least 1 Day This Week who Received a Completed COVID-19 Vaccination at Any Time</b> / <b>Number of All Healthcare Personnel Eligible to Work in this Facility for At Least 1
Day This Week) * 100</b>
<br>
* <mark> Recent Percentage of Current Healthcare Personnel Up to Date with COVID-19 Vaccines = The value of "Percentage of Current Healthcare Personnel who are Up to Date with COVID-19 Vaccines" for the current week, or if blank, for the prior week. Calculated as follows: <b> (Number of Healthcare Personnel Eligible to
Work in this Facility for At Least 1 Day This Week Up to Date with
COVID-19 Vaccines </b> / <b> Number of All Healthcare Personnel Eligible to
Work in this Facility for At Least 1 Day This Week) * 100 </b></mark>



### What I care about are the up-to-date metrics. This opens up a new quesiton: What does it mean to be up-to-date on COVID-19 vaccinations now? To have received all vax does plus all available boosters?

In [17]:
# subset cols
nh_latest_sub_2 = nh_latest_sub[[
                 'week_ending',
                 'federal_provider_number',
                 'provider_name',
                 'provider_address',
                 'provider_city',
                 'provider_state',
                 'provider_zip',
                 'provider_phone_number',
                 'county',
                 'submitted_data',
                 'data_passed_qa',
                 'res_wk_confirmed_c19',
                 'res_wk_all_deaths',
                 'res_wk_c19_deaths',
                 'num_all_beds',
                 'num_occ_beds',
                 'staff_wk_conf_c19',
                 'staff_wk_c19_deaths',
                 'short_nrsg_staff',
                 'short_aides',
                 'wk_res_confirmed_c19_per_1K',
                 'num_staff_new_pos_c19_test',
                 'num_res_new_flu',
                 'num_staff_pers_new_flu',
                 'can_test_all_res_staff_pers',
                 'num_res_in_facility',
                 'num_res_comp_c19_vax_anytime',
                 'num_res_decln_c19_vax_anytime',
                 'rec_pct_res_comp_c19_vax_anytime',
                 'pct_res_comp_c19_vax_anytime',
                 'num_res_utd_c19_vax',
                 'rec_pct_res_utd_c19_vax',
                 'num_hcp',
                 'num_hcp_comp_c19_vax_anytime',
                 'num_hcp_decln_c19_vax_anytime',
                 'rec_pct_hcp_comp_c19_vax_anytime',
                 'pct_hcp_comp_c19_vax_anytime',
                 'num_hcp_utd_c19_vax',
                 'rec_pct_hcp_utd_c19_vax',
                 'pct_hcp_utd_c19_vax',
                 'pct_hcp_comp_c19_vax_and_utd'
]]

nh_latest_sub_2.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd
0,2022-12-04,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,1,0,57,52,1,0,,,0,1,0,0,Y,53,49,4,,92.453,47,,81,70,10,,86.42,24,,29.63,34.286
1,2022-12-04,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,0,85,79,0,0,,,0,0,0,0,Y,80,73,5,,91.25,59,,186,152,30,,81.72,1,,0.538,0.658
2,2022-12-04,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,0,0,50,37,3,0,,,0,3,0,0,Y,38,38,0,,100,34,,81,77,4,,95.062,14,,17.284,18.182
3,2022-12-04,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,0,0,90,76,0,0,,,0,0,0,0,Y,76,74,1,,97.368,73,,102,99,2,,97.059,30,,29.412,30.303
4,2022-12-04,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,2,0,103,87,0,0,,,0,0,0,0,Y,89,82,5,,92.135,65,,141,117,18,,82.979,5,,3.546,4.274


In [18]:
nh_latest_sub_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184376 entries, 0 to 1184375
Data columns (total 41 columns):
 #   Column                            Non-Null Count    Dtype 
---  ------                            --------------    ----- 
 0   week_ending                       1184376 non-null  object
 1   federal_provider_number           1184376 non-null  object
 2   provider_name                     1184376 non-null  object
 3   provider_address                  1184376 non-null  object
 4   provider_city                     1184376 non-null  object
 5   provider_state                    1184376 non-null  object
 6   provider_zip                      1184376 non-null  object
 7   provider_phone_number             1184376 non-null  object
 8   county                            1184376 non-null  object
 9   submitted_data                    1184376 non-null  object
 10  data_passed_qa                    1184376 non-null  object
 11  res_wk_confirmed_c19              1184376 non-null

In [19]:
#replace spaces with null
nh_latest_sub_2=nh_latest_sub_2.replace(r'^\s*$', np.nan, regex=True)
nh_latest_sub_2.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd
0,2022-12-04,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,1,0,57,52,1,0,NaN,NaN,0,1,0,0,Y,53,49,4,NaN,92.453,47,NaN,81,70,10,NaN,86.42,24,NaN,29.63,34.286
1,2022-12-04,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,0,85,79,0,0,NaN,NaN,0,0,0,0,Y,80,73,5,NaN,91.25,59,NaN,186,152,30,NaN,81.72,1,NaN,0.538,0.658
2,2022-12-04,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,0,0,50,37,3,0,NaN,NaN,0,3,0,0,Y,38,38,0,NaN,100,34,NaN,81,77,4,NaN,95.062,14,NaN,17.284,18.182
3,2022-12-04,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,0,0,90,76,0,0,NaN,NaN,0,0,0,0,Y,76,74,1,NaN,97.368,73,NaN,102,99,2,NaN,97.059,30,NaN,29.412,30.303
4,2022-12-04,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,2,0,103,87,0,0,NaN,NaN,0,0,0,0,Y,89,82,5,NaN,92.135,65,NaN,141,117,18,NaN,82.979,5,NaN,3.546,4.274
5,2022-12-04,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,Limestone,Y,Y,0,0,0,149,132,0,0,NaN,NaN,0,0,0,0,Y,127,110,13,NaN,86.614,98,NaN,196,148,41,NaN,75.51,26,NaN,13.265,17.568
6,2022-12-04,015019,MERRY WOOD LODGE,280 MT HEBRON ROAD,ELMORE,AL,36025,3345678484,Elmore,Y,Y,0,2,0,124,99,0,0,N,N,0,0,0,0,Y,101,77,16,NaN,76.238,60,NaN,107,102,1,NaN,95.327,18,NaN,16.822,17.647
7,2022-12-04,015023,HATLEY HEALTH CARE INC,300 MEDICAL CENTER DRIVE,CLANTON,AL,35045,2057554960,Chilton,Y,Y,0,1,0,201,83,0,0,Y,Y,0,0,0,0,Y,84,83,0,NaN,98.81,34,NaN,130,113,15,NaN,86.923,74,NaN,56.923,65.487
8,2022-12-04,015024,"LIMESTONE NURSING AND REHABILITATION CENTER, LLC",1600 WEST HOBBS STREET,ATHENS,AL,35611,2562323461,Limestone,Y,Y,17,3,0,170,126,5,0,NaN,NaN,134.92,5,0,1,Y,130,104,21,NaN,80,44,NaN,193,171,18,NaN,88.601,12,NaN,6.218,7.018
9,2022-12-04,015027,"WETUMPKA HEALTH AND REHABILITATION, LLC",1825 HOLTVILLE ROAD,WETUMPKA,AL,36092,3345675131,Elmore,Y,Y,0,0,0,121,117,0,0,NaN,NaN,0,0,0,0,Y,156,97,25,NaN,62.179,53,NaN,182,147,26,NaN,80.769,2,NaN,1.099,1.361


In [20]:
#revise nums to float cols
float_cols = [
    'res_wk_confirmed_c19',
    'res_wk_all_deaths',
     'res_wk_c19_deaths',
     'num_all_beds',
     'num_occ_beds',
     'staff_wk_conf_c19',
     'staff_wk_c19_deaths',
     'wk_res_confirmed_c19_per_1K',
     'num_staff_new_pos_c19_test',
     'num_res_new_flu',
     'num_staff_pers_new_flu',
     'num_res_in_facility',
     'num_res_comp_c19_vax_anytime',
     'num_res_decln_c19_vax_anytime',
     'rec_pct_res_comp_c19_vax_anytime',
     'pct_res_comp_c19_vax_anytime',
     'num_res_utd_c19_vax',
     'rec_pct_res_utd_c19_vax',
     'num_hcp',
     'num_hcp_comp_c19_vax_anytime',
     'num_hcp_decln_c19_vax_anytime',
     'rec_pct_hcp_comp_c19_vax_anytime',
     'pct_hcp_comp_c19_vax_anytime',
     'num_hcp_utd_c19_vax',
     'rec_pct_hcp_utd_c19_vax',
     'pct_hcp_utd_c19_vax',
     'pct_hcp_comp_c19_vax_and_utd'
]

nh_latest_sub_2[float_cols]=nh_latest_sub_2[float_cols].astype('float')


In [21]:
nh_latest_sub_2['week_ending'] = pd.to_datetime(nh_latest_sub_2['week_ending'], format='%Y-%m-%d')

# subset to week ending greater than 7-2022
nh_latest_sub_end_22 = nh_latest_sub_2.loc[nh_latest_sub_2['week_ending']>='2022-07-10']
nh_latest_sub_end_22.info()

# create week start date col (should be a Mon)
nh_latest_sub_end_22['week_start'] = nh_latest_sub_end_22['week_ending'] - timedelta(days=6)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 363100 entries, 0 to 1184375
Data columns (total 41 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   week_ending                       363100 non-null  datetime64[ns]
 1   federal_provider_number           363100 non-null  object        
 2   provider_name                     363100 non-null  object        
 3   provider_address                  363100 non-null  object        
 4   provider_city                     363100 non-null  object        
 5   provider_state                    363100 non-null  object        
 6   provider_zip                      363100 non-null  object        
 7   provider_phone_number             363100 non-null  object        
 8   county                            363100 non-null  object        
 9   submitted_data                    363100 non-null  object        
 10  data_passed_qa                 

In [22]:
nh_latest_sub_end_22.sort_values(by = ['federal_provider_number','week_ending']).head(100)


,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start
866600,2022-07-10,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,51.0,0.0,0.0,NaN,NaN,0.00,0.0,0.0,0.0,Y,51.0,48.0,2.0,NaN,94.118,44.0,NaN,81.0,71.0,9.0,NaN,87.654,21.0,NaN,25.926,29.577,2022-07-04
881748,2022-07-17,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,53.0,1.0,0.0,NaN,NaN,0.00,1.0,0.0,0.0,Y,53.0,51.0,1.0,NaN,96.226,33.0,NaN,81.0,70.0,10.0,NaN,86.420,24.0,NaN,29.630,34.286,2022-07-11
896892,2022-07-24,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,54.0,0.0,0.0,NaN,NaN,0.00,0.0,0.0,0.0,Y,54.0,52.0,1.0,NaN,96.296,47.0,NaN,80.0,70.0,9.0,NaN,87.500,34.0,NaN,42.500,48.571,2022-07-18
912035,2022-07-31,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,53.0,0.0,0.0,NaN,NaN,0.00,0.0,0.0,0.0,Y,54.0,52.0,1.0,NaN,96.296,35.0,NaN,81.0,70.0,10.0,NaN,86.420,21.0,NaN,25.926,30.000,2022-07-25
927178,2022-08-07,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,1.0,0.0,57.0,52.0,0.0,0.0,NaN,NaN,0.00,0.0,0.0,0.0,Y,53.0,51.0,1.0,NaN,96.226,35.0,NaN,81.0,70.0,11.0,NaN,86.420,26.0,NaN,32.099,37.143,2022-08-01
942321,2022-08-14,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,1.0,0.0,57.0,51.0,1.0,0.0,NaN,NaN,0.00,1.0,0.0,0.0,Y,54.0,52.0,2.0,NaN,96.296,40.0,NaN,81.0,72.0,9.0,NaN,88.889,26.0,NaN,32.099,36.111,2022-08-08
957460,2022-08-21,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,1.0,0.0,57.0,54.0,0.0,0.0,NaN,NaN,0.00,0.0,0.0,0.0,Y,54.0,51.0,3.0,NaN,94.444,29.0,NaN,81.0,70.0,11.0,NaN,86.420,12.0,NaN,14.815,17.143,2022-08-15
972597,2022-08-28,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,56.0,1.0,0.0,NaN,NaN,0.00,1.0,0.0,0.0,Y,54.0,51.0,3.0,NaN,94.444,29.0,NaN,81.0,70.0,11.0,NaN,86.420,12.0,NaN,14.815,17.143,2022-08-22
987732,2022-09-04,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,1.0,1.0,0.0,57.0,55.0,1.0,0.0,NaN,NaN,18.18,1.0,0.0,0.0,Y,54.0,51.0,3.0,NaN,94.444,33.0,NaN,81.0,71.0,10.0,NaN,87.654,21.0,NaN,25.926,29.577,2022-08-29
1002865,2022-09-11,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,3.0,0.0,0.0,57.0,53.0,2.0,0.0,NaN,NaN,56.60,2.0,0.0,0.0,Y,54.0,51.0,3.0,NaN,94.444,29.0,NaN,81.0,70.0,11.0,NaN,86.420,12.0,NaN,14.815,17.143,2022-09-05


In [23]:
nh_latest_sub_end_22['month_year'] = pd.to_datetime(nh_latest_sub_end_22['week_start']).dt.to_period('M')
nh_latest_sub_end_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year
0,2022-12-04,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,1.0,0.0,57.0,52.0,1.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,Y,53.0,49.0,4.0,NaN,92.453,47.0,NaN,81.0,70.0,10.0,NaN,86.420,24.0,NaN,29.630,34.286,2022-11-28,2022-11
1,2022-12-04,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,79.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,80.0,73.0,5.0,NaN,91.250,59.0,NaN,186.0,152.0,30.0,NaN,81.720,1.0,NaN,0.538,0.658,2022-11-28,2022-11
2,2022-12-04,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,0.0,0.0,50.0,37.0,3.0,0.0,NaN,NaN,0.0,3.0,0.0,0.0,Y,38.0,38.0,0.0,NaN,100.000,34.0,NaN,81.0,77.0,4.0,NaN,95.062,14.0,NaN,17.284,18.182,2022-11-28,2022-11
3,2022-12-04,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,0.0,0.0,90.0,76.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,76.0,74.0,1.0,NaN,97.368,73.0,NaN,102.0,99.0,2.0,NaN,97.059,30.0,NaN,29.412,30.303,2022-11-28,2022-11
4,2022-12-04,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,2.0,0.0,103.0,87.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,89.0,82.0,5.0,NaN,92.135,65.0,NaN,141.0,117.0,18.0,NaN,82.979,5.0,NaN,3.546,4.274,2022-11-28,2022-11


In [24]:
nh_latest_sub_end_22['rev_pct_res_utd_c19_vax'] = nh_latest_sub_end_22.num_res_utd_c19_vax/nh_latest_sub_end_22.num_res_in_facility
nh_latest_sub_end_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year,rev_pct_res_utd_c19_vax
0,2022-12-04,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,1.0,0.0,57.0,52.0,1.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,Y,53.0,49.0,4.0,NaN,92.453,47.0,NaN,81.0,70.0,10.0,NaN,86.420,24.0,NaN,29.630,34.286,2022-11-28,2022-11,0.886792
1,2022-12-04,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,79.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,80.0,73.0,5.0,NaN,91.250,59.0,NaN,186.0,152.0,30.0,NaN,81.720,1.0,NaN,0.538,0.658,2022-11-28,2022-11,0.737500
2,2022-12-04,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,0.0,0.0,50.0,37.0,3.0,0.0,NaN,NaN,0.0,3.0,0.0,0.0,Y,38.0,38.0,0.0,NaN,100.000,34.0,NaN,81.0,77.0,4.0,NaN,95.062,14.0,NaN,17.284,18.182,2022-11-28,2022-11,0.894737
3,2022-12-04,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,0.0,0.0,90.0,76.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,76.0,74.0,1.0,NaN,97.368,73.0,NaN,102.0,99.0,2.0,NaN,97.059,30.0,NaN,29.412,30.303,2022-11-28,2022-11,0.960526
4,2022-12-04,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,2.0,0.0,103.0,87.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,89.0,82.0,5.0,NaN,92.135,65.0,NaN,141.0,117.0,18.0,NaN,82.979,5.0,NaN,3.546,4.274,2022-11-28,2022-11,0.730337


In [25]:
nh_latest_sub_end_22['rev_pct_hcp_utd_c19_vax'] = nh_latest_sub_end_22.num_hcp_utd_c19_vax/nh_latest_sub_end_22.num_hcp
nh_latest_sub_end_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year,rev_pct_res_utd_c19_vax,rev_pct_hcp_utd_c19_vax
0,2022-12-04,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,1.0,0.0,57.0,52.0,1.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,Y,53.0,49.0,4.0,NaN,92.453,47.0,NaN,81.0,70.0,10.0,NaN,86.420,24.0,NaN,29.630,34.286,2022-11-28,2022-11,0.886792,0.296296
1,2022-12-04,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,79.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,80.0,73.0,5.0,NaN,91.250,59.0,NaN,186.0,152.0,30.0,NaN,81.720,1.0,NaN,0.538,0.658,2022-11-28,2022-11,0.737500,0.005376
2,2022-12-04,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,0.0,0.0,50.0,37.0,3.0,0.0,NaN,NaN,0.0,3.0,0.0,0.0,Y,38.0,38.0,0.0,NaN,100.000,34.0,NaN,81.0,77.0,4.0,NaN,95.062,14.0,NaN,17.284,18.182,2022-11-28,2022-11,0.894737,0.172840
3,2022-12-04,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,0.0,0.0,90.0,76.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,76.0,74.0,1.0,NaN,97.368,73.0,NaN,102.0,99.0,2.0,NaN,97.059,30.0,NaN,29.412,30.303,2022-11-28,2022-11,0.960526,0.294118
4,2022-12-04,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,2.0,0.0,103.0,87.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,89.0,82.0,5.0,NaN,92.135,65.0,NaN,141.0,117.0,18.0,NaN,82.979,5.0,NaN,3.546,4.274,2022-11-28,2022-11,0.730337,0.035461


In [26]:
nh_latest_sub_end_22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363100 entries, 0 to 1184375
Data columns (total 45 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   week_ending                       363100 non-null  datetime64[ns]
 1   federal_provider_number           363100 non-null  object        
 2   provider_name                     363100 non-null  object        
 3   provider_address                  363100 non-null  object        
 4   provider_city                     363100 non-null  object        
 5   provider_state                    363100 non-null  object        
 6   provider_zip                      363100 non-null  object        
 7   provider_phone_number             363100 non-null  object        
 8   county                            363100 non-null  object        
 9   submitted_data                    363100 non-null  object        
 10  data_passed_qa                 

In [27]:
nh_latest_sub_end_22.data_passed_qa.value_counts()

Y    356583
N      2940
Name: data_passed_qa, dtype: int64

In [28]:
nh_latest_sub_end_22.submitted_data.value_counts()

Y    359523
N      3577
Name: submitted_data, dtype: int64

In [62]:
nh_latest_sub_end_22=nh_latest_sub_end_22.sort_values(by='week_start')

### subset to nelson county, ky

In [58]:
nh_latest_sub_end_22_nels_ky = nh_latest_sub_end_22.loc[(nh_latest_sub_end_22['county'] == 'Nelson') & (nh_latest_sub_end_22['provider_state'] == 'KY')].sort_values(by='week_start')
nh_latest_sub_end_22_nels_ky.style.set_sticky(axis='columns')


,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year,rev_pct_res_utd_c19_vax,rev_pct_hcp_utd_c19_vax
871549,2022-07-10 00:00:00,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,0.000000,0.000000,0.000000,100.000000,81.000000,4.000000,0.000000,nan,nan,0.000000,4.000000,0.000000,0.000000,Y,81.000000,68.000000,11.000000,nan,83.951000,1.000000,nan,65.000000,51.000000,14.000000,nan,78.462000,2.000000,nan,3.077000,3.922000,2022-07-04 00:00:00,2022-07,0.012346,0.030769
871688,2022-07-10 00:00:00,185342,SIGNATURE HEALTHCARE AT COLONIAL REHAB & WELLNESS,708 BARTLEY AVENUE,BARDSTOWN,KY,40004,5023489260,Nelson,Y,Y,3.000000,0.000000,0.000000,65.000000,53.000000,1.000000,0.000000,nan,nan,56.600000,1.000000,0.000000,0.000000,Y,53.000000,48.000000,5.000000,nan,90.566000,29.000000,nan,176.000000,79.000000,9.000000,nan,44.886000,31.000000,nan,17.614000,39.241000,2022-07-04 00:00:00,2022-07,0.547170,0.176136
871731,2022-07-10 00:00:00,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,nan,nan,nan,0.000000,0.000000,0.000000,Y,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,2022-07-04 00:00:00,2022-07,nan,nan
886696,2022-07-17 00:00:00,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,0.000000,0.000000,0.000000,100.000000,86.000000,1.000000,0.000000,nan,nan,0.000000,1.000000,0.000000,0.000000,Y,86.000000,71.000000,13.000000,nan,82.558000,1.000000,nan,65.000000,51.000000,13.000000,nan,78.462000,2.000000,nan,3.077000,3.922000,2022-07-11 00:00:00,2022-07,0.011628,0.030769
886835,2022-07-17 00:00:00,185342,SIGNATURE HEALTHCARE AT COLONIAL REHAB & WELLNESS,708 BARTLEY AVENUE,BARDSTOWN,KY,40004,5023489260,Nelson,Y,Y,0.000000,0.000000,0.000000,65.000000,51.000000,4.000000,0.000000,nan,nan,0.000000,4.000000,0.000000,0.000000,Y,51.000000,46.000000,5.000000,nan,90.196000,28.000000,nan,184.000000,82.000000,10.000000,nan,44.565000,33.000000,nan,17.935000,40.244000,2022-07-11 00:00:00,2022-07,0.549020,0.179348
886878,2022-07-17 00:00:00,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,nan,nan,nan,0.000000,0.000000,0.000000,Y,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,2022-07-11 00:00:00,2022-07,nan,nan
902021,2022-07-24 00:00:00,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,nan,nan,nan,0.000000,0.000000,0.000000,Y,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,2022-07-18 00:00:00,2022-07,nan,nan
901978,2022-07-24 00:00:00,185342,SIGNATURE HEALTHCARE AT COLONIAL REHAB & WELLNESS,708 BARTLEY AVENUE,BARDSTOWN,KY,40004,5023489260,Nelson,Y,Y,0.000000,

In [59]:
nh_latest_sub_end_22_nels_ky.provider_name.value_counts()

LANDMARK OF BARDSTOWN REHABILITATION AND NURSING     24
SIGNATURE HEALTHCARE AT COLONIAL REHAB & WELLNESS    24
FLAGET MEMORIAL HOSPITAL NF                          24
Name: provider_name, dtype: int64

In [60]:
nh_latest_sub_end_22_nels_ky.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 871549 to 35153
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   week_ending                       72 non-null     datetime64[ns]
 1   federal_provider_number           72 non-null     object        
 2   provider_name                     72 non-null     object        
 3   provider_address                  72 non-null     object        
 4   provider_city                     72 non-null     object        
 5   provider_state                    72 non-null     object        
 6   provider_zip                      72 non-null     object        
 7   provider_phone_number             72 non-null     object        
 8   county                            72 non-null     object        
 9   submitted_data                    72 non-null     object        
 10  data_passed_qa                    71 non-nul

### plot time series of staff up to date c19 vax and residents up to date c19 vax

In [64]:
fig = px.line(nh_latest_sub_end_22.loc[(nh_latest_sub_end_22['county'] == 'Nelson') & (nh_latest_sub_end_22['provider_state'] == 'KY') & (nh_latest_sub_end_22['data_passed_qa'] == 'Y')]
                 , x="week_start", y=["rev_pct_res_utd_c19_vax","rev_pct_hcp_utd_c19_vax"],
                 #color="rev_pct_res_utd_c19_vax",
                 facet_col="provider_name",facet_col_wrap=1)
#fig.update_yaxes(matches=None)
fig.update_layout(
    autosize=True,
    width=1000,
    height=1000,   
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')



for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
        
fig.show()


ERROR! Session/line number was not unique in database. History logging moved to new session 467


In [53]:
nh_latest_sub_end_22_nels_ky.loc[nh_latest_sub_end_22_nels_ky['provider_name']=='FLAGET MEMORIAL HOSPITAL NF']


,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year,rev_pct_res_utd_c19_vax,rev_pct_hcp_utd_c19_vax
5119,2022-12-04,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,Y,0.0,0.0,0.0,12.0,4.0,0.0,0.0,NaN,NaN,0.00,0.0,0.0,0.0,Y,7.0,6.0,1.0,NaN,85.714,2.0,NaN,79.0,70.0,9.0,NaN,88.608,10.0,NaN,12.658,14.286,2022-11-28,2022-11,0.285714,0.126582
20227,2022-12-11,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,Y,0.0,0.0,0.0,12.0,2.0,0.0,0.0,NaN,NaN,0.00,0.0,0.0,0.0,Y,8.0,5.0,3.0,NaN,62.500,2.0,NaN,79.0,70.0,9.0,NaN,88.608,10.0,NaN,12.658,14.286,2022-12-05,2022-12,0.250000,0.126582
35335,2022-12-18,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,Y,0.0,0.0,0.0,12.0,4.0,0.0,0.0,NaN,NaN,0.00,0.0,0.0,0.0,Y,NaN,NaN,NaN,62.5,NaN,NaN,25.0,NaN,NaN,NaN,88.608,NaN,NaN,12.658,NaN,NaN,2022-12-12,2022-12,NaN,NaN
871731,2022-07-10,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.0,0.0,0.0,12.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Y,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.000,0.000,2022-07-04,2022-07,NaN,NaN
886878,2022-07-17,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.0,0.0,0.0,12.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Y,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.000,0.000,2022-07-11,2022-07,NaN,NaN
902021,2022-07-24,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.0,0.0,0.0,12.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Y,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.000,0.000,2022-07-18,2022-07,NaN,NaN
917163,2022-07-31,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.0,0.0,0.0,12.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Y,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.000,0.000,2022-07-25,2022-07,NaN,NaN
932306,2022-08-07,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.0,0.0,0.0,12.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Y,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.000,0.000,2022-08-01,2022-08,NaN,NaN
947446,2022-08-14,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.0,0.0,0.0,12.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Y,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.000,0.000,2022-08-08,2022-08,NaN,NaN
962584,2022-08-21,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.0,0.0,0.0,12.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Y,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.0,0.0,0.0,NaN,0.000,0.0,NaN,0.000,0.000,2022-08-15,2022-08,NaN,NaN


In [58]:
# df.groupby(['Years','Gender','Country'])[['Salary','Work ex']].mean()
# df.reset_index(inpalce=True) 

nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='Y'].num_res_utd_c19_vax.sum()


13934783.0

In [59]:
nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='Y'].num_res_in_facility.sum()



28657424.0

In [74]:
nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='Y'].num_hcp.sum()


47829217.0

In [75]:
nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='Y'].num_hcp_utd_c19_vax.sum()

15765757.0

In [60]:
num_res_utd_c19_vax_sum/num_res_in_facility_sum

0.4862538586859726

### Group to city level

In [68]:
nh_latest_sub_end_22_city_grp=nh_latest_sub_end_22.groupby(['week_start','provider_state','county','provider_city','data_passed_qa'],dropna=False)['num_res_in_facility','num_res_utd_c19_vax','num_hcp','num_hcp_utd_c19_vax'].sum().reset_index()
nh_latest_sub_end_22_city_grp.head()

ERROR! Session/line number was not unique in database. History logging moved to new session 471


,week_start,provider_state,county,provider_city,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax
0,2022-07-04,AK,Anchorage,ANCHORAGE,Y,237.0,188.0,434.0,201.0
1,2022-07-04,AK,Bethel,BETHEL,Y,14.0,10.0,42.0,35.0
2,2022-07-04,AK,Cordova-Mccarthy,CORDOVA,Y,10.0,6.0,21.0,17.0
3,2022-07-04,AK,Fairbanks,FAIRBANKS,Y,70.0,61.0,154.0,74.0
4,2022-07-04,AK,Juneau,JUNEAU,Y,48.0,29.0,125.0,75.0


In [78]:
nh_latest_sub_end_22_city_grp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175554 entries, 0 to 175553
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   week_ending          175554 non-null  datetime64[ns]
 1   provider_state       175554 non-null  object        
 2   county               175554 non-null  object        
 3   provider_city        175554 non-null  object        
 4   data_passed_qa       172247 non-null  object        
 5   num_res_in_facility  175554 non-null  float64       
 6   num_res_utd_c19_vax  175554 non-null  float64       
 7   num_hcp              175554 non-null  float64       
 8   num_hcp_utd_c19_vax  175554 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 12.1+ MB


In [79]:
nh_latest_sub_end_22_city_grp.loc[nh_latest_sub_end_22_city_grp['data_passed_qa']=='Y'].num_res_utd_c19_vax.sum()


13934783.0

In [80]:
nh_latest_sub_end_22_city_grp.loc[nh_latest_sub_end_22_city_grp['data_passed_qa']=='Y'].num_res_in_facility.sum()


28657424.0

In [81]:
nh_latest_sub_end_22_city_grp.loc[nh_latest_sub_end_22_city_grp['data_passed_qa']=='Y'].num_hcp.sum()


47829217.0

In [82]:
nh_latest_sub_end_22_city_grp.loc[nh_latest_sub_end_22_city_grp['data_passed_qa']=='Y'].num_hcp_utd_c19_vax.sum()


15765757.0

In [83]:
# create pct avg cols
nh_latest_sub_end_22_city_grp['pct_res_utd_c19_vax'] = nh_latest_sub_end_22_city_grp['num_res_utd_c19_vax']/nh_latest_sub_end_22_city_grp['num_res_in_facility']
nh_latest_sub_end_22_city_grp['pct_hcp_utd_c19_vax'] = nh_latest_sub_end_22_city_grp['num_hcp_utd_c19_vax']/nh_latest_sub_end_22_city_grp['num_hcp']
nh_latest_sub_end_22_city_grp.head()

,week_ending,provider_state,county,provider_city,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax,pct_res_utd_c_19_vax
0,2022-07-10,AK,Anchorage,ANCHORAGE,Y,237.0,188.0,434.0,201.0,0.793249
1,2022-07-10,AK,Bethel,BETHEL,Y,14.0,10.0,42.0,35.0,0.714286
2,2022-07-10,AK,Cordova-Mccarthy,CORDOVA,Y,10.0,6.0,21.0,17.0,0.600000
3,2022-07-10,AK,Fairbanks,FAIRBANKS,Y,70.0,61.0,154.0,74.0,0.871429
4,2022-07-10,AK,Juneau,JUNEAU,Y,48.0,29.0,125.0,75.0,0.604167


### Group to county level

In [69]:
nh_latest_sub_end_22_county_grp=nh_latest_sub_end_22.groupby(['week_start','provider_state','county','data_passed_qa'],dropna=False)['num_res_in_facility','num_res_utd_c19_vax','num_hcp','num_hcp_utd_c19_vax'].sum().reset_index()
nh_latest_sub_end_22_county_grp.head()

,week_start,provider_state,county,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax
0,2022-07-04,AK,Anchorage,Y,237.0,188.0,434.0,201.0
1,2022-07-04,AK,Bethel,Y,14.0,10.0,42.0,35.0
2,2022-07-04,AK,Cordova-Mccarthy,Y,10.0,6.0,21.0,17.0
3,2022-07-04,AK,Fairbanks,Y,70.0,61.0,154.0,74.0
4,2022-07-04,AK,Juneau,Y,48.0,29.0,125.0,75.0


In [78]:
nh_latest_sub_end_22_county_grp.loc[nh_latest_sub_end_22_county_grp['data_passed_qa']=='Y'].num_res_utd_c19_vax.sum()


13934783.0

In [70]:
# create pct avg cols
nh_latest_sub_end_22_county_grp['pct_res_utd_c19_vax']= nh_latest_sub_end_22_county_grp['num_res_utd_c19_vax']/nh_latest_sub_end_22_county_grp['num_res_in_facility']
nh_latest_sub_end_22_county_grp['pct_hcp_utd_c19_vax'] = nh_latest_sub_end_22_county_grp['num_hcp_utd_c19_vax']/nh_latest_sub_end_22_county_grp['num_hcp']
nh_latest_sub_end_22_county_grp.head()


,week_start,provider_state,county,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax,pct_res_utd_c19_vax,pct_hcp_utd_c19_vax
0,2022-07-04,AK,Anchorage,Y,237.0,188.0,434.0,201.0,0.793249,0.463134
1,2022-07-04,AK,Bethel,Y,14.0,10.0,42.0,35.0,0.714286,0.833333
2,2022-07-04,AK,Cordova-Mccarthy,Y,10.0,6.0,21.0,17.0,0.600000,0.809524
3,2022-07-04,AK,Fairbanks,Y,70.0,61.0,154.0,74.0,0.871429,0.480519
4,2022-07-04,AK,Juneau,Y,48.0,29.0,125.0,75.0,0.604167,0.600000


In [77]:
fig = px.line(nh_latest_sub_end_22_county_grp.loc[(nh_latest_sub_end_22_county_grp['provider_state'] == 'KY') &
                                                  (nh_latest_sub_end_22_county_grp['county'] == 'Nelson') &
                                                  (nh_latest_sub_end_22_county_grp['data_passed_qa'] == 'Y')],
                 x="week_start", y=["pct_res_utd_c19_vax","pct_hcp_utd_c19_vax"],
                 #facet_col="county", facet_col_wrap=7,
                  title="Nursing home residents and staff Covid-19 vaccination rates: Nelson County KY"
             )
#fig.update_yaxes(matches=None)
fig.update_layout(
    #autosize=True,
    #width=1000,
    #height=600,   
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')



for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
        
fig.show()

### Group to state level

In [79]:
nh_latest_sub_end_22_state_grp=nh_latest_sub_end_22.groupby(['week_start','provider_state','data_passed_qa'],dropna=False)['num_res_in_facility','num_res_utd_c19_vax','num_hcp','num_hcp_utd_c19_vax'].sum().reset_index()
nh_latest_sub_end_22_state_grp.head()


ERROR! Session/line number was not unique in database. History logging moved to new session 476


,week_start,provider_state,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax
0,2022-07-04,AK,Y,720.0,441.0,2132.0,966.0
1,2022-07-04,AL,Y,21048.0,9861.0,33118.0,8309.0
2,2022-07-04,AR,Y,15640.0,8662.0,30756.0,14313.0
3,2022-07-04,AZ,N,383.0,0.0,715.0,135.0
4,2022-07-04,AZ,Y,11047.0,3613.0,20802.0,6149.0


In [80]:
# create pct avg cols
nh_latest_sub_end_22_state_grp['pct_res_utd_c19_vax']= nh_latest_sub_end_22_state_grp['num_res_utd_c19_vax']/nh_latest_sub_end_22_state_grp['num_res_in_facility']
nh_latest_sub_end_22_state_grp['pct_hcp_utd_c19_vax'] = nh_latest_sub_end_22_state_grp['num_hcp_utd_c19_vax']/nh_latest_sub_end_22_state_grp['num_hcp']
nh_latest_sub_end_22_state_grp.head()


,week_start,provider_state,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax,pct_res_utd_c19_vax,pct_hcp_utd_c19_vax
0,2022-07-04,AK,Y,720.0,441.0,2132.0,966.0,0.612500,0.453096
1,2022-07-04,AL,Y,21048.0,9861.0,33118.0,8309.0,0.468501,0.250891
2,2022-07-04,AR,Y,15640.0,8662.0,30756.0,14313.0,0.553836,0.465373
3,2022-07-04,AZ,N,383.0,0.0,715.0,135.0,0.000000,0.188811
4,2022-07-04,AZ,Y,11047.0,3613.0,20802.0,6149.0,0.327057,0.295597


In [81]:
fig = px.line(nh_latest_sub_end_22_state_grp.loc[(nh_latest_sub_end_22_state_grp['provider_state'] == 'KY') &
                                                  (nh_latest_sub_end_22_state_grp['data_passed_qa'] == 'Y')],
                 x="week_start", y=["pct_res_utd_c19_vax","pct_hcp_utd_c19_vax"],
                 #facet_col="county", facet_col_wrap=7,
                  title="Nursing home residents and staff Covid-19 vaccination rates: Nelson County KY"
             )
#fig.update_yaxes(matches=None)
fig.update_layout(
    #autosize=True,
    #width=1000,
    #height=600,   
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')



for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
        
fig.show()

### What pct of entire dataset passed qa?

In [ ]:
#Entire dataset?
nh_latest_sub_end_22['data_passed_qa'].value_counts(normalize=True)

In [ ]:
#By State?
state_qa=nh_latest_sub_end_22.groupby('provider_state')['data_passed_qa'].value_counts(normalize=True).to_frame(name='pct').reset_index()

state_qa.head()

In [ ]:
#Sorting states by qa failed rate descending
state_qa.loc[state_qa['data_passed_qa']=='N'].sort_values(by=['pct'], ascending=False)


In [ ]:
#By State and County?
state_county_qa=nh_latest_sub_end_22.groupby(['provider_state','county'])['data_passed_qa'].value_counts(normalize=True).to_frame(name='pct').reset_index()
state_county_qa.head()


In [ ]:
#Sorting counties by qa failed rate, state descending
state_county_qa.loc[state_county_qa['data_passed_qa']=='N'].sort_values(by=['pct','provider_state'], ascending=[False,True])

In [ ]:
#Sorting counties by state, qa failed rate descending
state_county_qa.loc[state_county_qa['data_passed_qa']=='N'].sort_values(by=(['provider_state','pct']), ascending=[True,False])
                                                                                                                  

In [ ]:
nh_dq_fail=nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='N']
nh_dq_fail.info()

In [ ]:
nh_dq_fail.provider_name.value_counts().sort_index()

In [ ]:
nh_dq_pass=nh_sub.loc[nh_sub['data_passed_qa']=='Y']
nh_dq_pass.head()

In [ ]:
nh_dq_pass.tail()

In [ ]:
nh_dq_pass.info()

In [ ]:
#subset to ky
nh_ky=nh_sub.loc[nh_sub['provider_state']=='KY']
nh_ky.head()

In [ ]:
# who failed dq check and how many for how many occurances (weeks)?
nh_ky.loc[nh_ky['data_passed_qa']=='N'].provider_name.value_counts()

In [ ]:
nh_ky_pass=nh_ky.loc[nh_ky['data_passed_qa']=='Y']
nh_ky_pass.info()

In [ ]:
# subset to jefferson county
jeff_cty_pass=nh_ky_pass.loc[(nh_ky_pass['month_year']>='2021-07') & (nh_ky_pass['data_passed_qa']=='Y') & (nh_ky_pass['county']=='Jefferson')].sort_values(by=['provider_name','week_ending'])

#sort by facility, week ending date, num_res_in_facility
jeff_cty_pass.sort_values(['federal_provider_number','week_ending','num_res_in_facility'])


In [ ]:
jeff_cty_pass.loc[jeff_cty_pass['num_res_in_facility'].isnull()].sort_values('week_ending')

In [ ]:
jeff_cty_pass.federal_provider_number.value_counts()

In [ ]:
# Why does 185301 only have 10 occurrences? 

jeff_cty_pass.loc[jeff_cty_pass['federal_provider_number'] == '185301']

# Regis Woods. Shut down sept 2021 for health and safety violations: 
# https://www.courier-journal.com/story/news/local/2021/09/15/louisville-nursing-home-regis-woods-losing-medicaid-funding-residents-relocated/8349023002/


In [ ]:
# Regis Woods. Shut down sept 2021 for health and safety violations: 
# https://www.courier-journal.com/story/news/local/2021/09/15/louisville-nursing-home-regis-woods-losing-medicaid-funding-residents-relocated/8349023002/


### What did Jefferson county nh staff vax rates and res vax rates look like over the past year? (Median and Mean)


In [ ]:
jeff_cty_pass_avg_med=jeff_cty_pass.groupby(['month_year','county'],dropna=False).agg(
sum_hcp_comp_c19_vax=('num_hcp_comp_c19_vax','sum'),
sum_hcp=('num_hcp','sum'),
sum_res_comp_c19_vax=('num_res_comp_c19_vax','sum'),
sum_res_in_facility=('num_res_in_facility','sum'),
pct_hcp_comp_c19_vax_median=('pct_hcp_comp_c19_vax','median'),
pct_res_comp_c19_vax_median=('pct_res_comp_c19_vax','median')).reset_index()


In [ ]:
jeff_cty_pass_avg_med['pct_hcp_comp_c19_vax_avg'] = jeff_cty_pass_avg_med['sum_hcp_comp_c19_vax']/jeff_cty_pass_avg_med['sum_hcp']*100
jeff_cty_pass_avg_med['pct_res_comp_c19_vax_avg'] = jeff_cty_pass_avg_med['sum_res_comp_c19_vax']/jeff_cty_pass_avg_med['sum_res_in_facility']*100


In [ ]:
jeff_cty_pass_avg_med

### Plot: distribution of staff vax and res vax pcts for Jefferson county by month-yr

In [ ]:
jeff_cty_pass['month_year'] = pd.to_datetime(jeff_cty_pass['month_year'], format='%Y-%m')


fig = px.box(jeff_cty_pass, x='month_year', y='pct_hcp_comp_c19_vax', points='all')

fig.update_layout(
    autosize=False,
    width=1200,
    height=600
)

fig.show()

### WIP see below

In [ ]:
## box plot
jeff_cty_pass_fig = px.box(
    jeff_cty_pass, 
    x = pd.to_datetime('week_start'),
    y = 'pct_hcp_comp_c19_vax',
    title = 'Pct healthcare staff who completed covid-19 vaccination',
    points = 'all', # will display dots next to the boxes 
    labels = {
        'x': pd.to_datetime('week_start'),
        'y': 'pct_hcp_comp_c19_vax'
    }
)

jeff_cty_pass_fig.update_layout(
    plot_bgcolor = 'white',
    xaxis = dict(
        showline = True,
        linecolor = 'black'
    ),
    
    yaxis = dict(
        showline = True,
        showgrid = False,
        linecolor = 'black'
    )
    
)

jeff_cty_pass_fig.show()

### Plot time series line for Jefferson county by month

## nelson county subset

In [ ]:
# subset to nelson county, ky
nh_ky_cty_nelson = nh_ky.loc[nh_ky.county=='Nelson'].reset_index(drop=True)
nh_ky_cty_nelson.sort_values(by=['provider_name','week_ending'])

In [ ]:
nh_ky_cty_nelson.info()

In [ ]:
nh_ky_cty_nelson.provider_name.value_counts()

In [ ]:
nh_ky_cty_nelson.loc[(nh_ky_cty_nelson['month_year']>='2021-10') & (nh_ky_cty_nelson['data_passed_qa']=='Y')].sort_values(by=['provider_name','week_ending'])

In [ ]:
nh_ky_cty_nelson.loc[(nh_ky_cty_nelson['month_year']>='2021-10') & (nh_ky_cty_nelson['data_passed_qa']=='Y')].quantile([0, .1, .25, .33, .5, .67, .75, .9, 1])

In [ ]:
nh_ky_cty_nelson.loc[(nh_ky_cty_nelson['month_year']>='2022-04') & (nh_ky_cty_nelson['data_passed_qa']=='Y')].pct_hcp_comp_c19_vax.mean()

In [ ]:
nh_ky_cty_nelson.loc[(nh_ky_cty_nelson['month_year']>='2022-04') & (nh_ky_cty_nelson['data_passed_qa']=='Y')].pct_hcp_comp_c19_vax.median()

## Shapefile data

In [ ]:
# Read in shp file as dataframe
# Nursing home shapefile data source: HIFLD data retrieved from: 
# https://hifld-geoplatform.opendata.arcgis.com/datasets/geoplatform::nursing-homes/about
# Latest update for files: Jun 29, 22

nh_shp_df = gpd.read_file('data/Nursing_Homes_shape/NursingHomes.shp')
nh_shp_df.head()

In [ ]:
nh_shp_df.info()

In [ ]:
nh_shp_df.loc[nh_shp_df['NAME']=='CHEROKEE PARK REHABILITATION']